## Workflow: **Parallelization**

Parallelization takes advantage of tasks that can broken up into discrete independent parts.

The user's prompt is passed to multiple LLMs simultaneously. Once all the LLMs respond, their answers are all sent to a final LLM call to be aggregated for the final answer.

![parallelization](resources/parallelization.jpg)

## **Use Cases:**

- Using one LLM to answer a user's question, while at the same time using another to screen the question for inappropriate content or requests.
- Reviewing a piece of code for both security vulnerabilities and stylistic improvements at the same time.
- Analyzing a lengthy document by dividing it into sections and assigning each section to a separate LLM for summarization, then combining the summaries into a comprehensive overview.
- Simultaneously analyzing a text for emotional tone, intent, and potential biases, with each aspect handled by a dedicated LLM.
- Translating a document into multiple languages at the same time by assigning each language to a separate LLM, then aggregating the results for multilingual output.

In [ ]:
%pip install openai pydantic nest-asyncio --upgrade

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import asyncio
from typing import Literal
from pydantic import BaseModel
from openai import AsyncOpenAI

In [4]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [6]:
client = AsyncOpenAI()
MODEL = "gpt-4o-mini"

In [7]:
async def analyze_decision(role: str, context: str) -> str:
    prompt = f'''
    Act as a {role}.
    You are analysing the potential direction of a user, like a career coach. Your goal is to analyse the {context}.
    Then to determine the right way for them to move forward. It's critical that you provide a helpful answer about how the user should move forward.
    '''
    response = await client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=1000,
    )
    if response.choices[0].message.content is None:
        return "No answer"
    return response.choices[0].message.content

async def aggregate_to_a_final_answer(role: str, context: str, answers: list[str]) -> str:
    prompt = f'''
    Act as a {role}.
    You are analysing the potential direction of a user, like a career coach. Your goal is to analyse the {context}.
    Then to determine the right way for them to move forward. It's critical that you provide a helpful answer about how the user should move forward.
    You have also received previous answers from {len(answers)} different agents.
    Here are the answers:
    {answers}
    Your job is to aggregate the answers and provide a final answer.
    '''
    response = await client.beta.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=1000,
    )
    if response.choices[0].message.content is None:
        return "No answer"
    return response.choices[0].message.content

In [8]:
async def main():
    context = """I'm looking to start a new project either building an AI blog posting assistant as a SaaS or building a keyboard shortcut application as a desktop app.
    Here is my previous experience:
    - I'm a software engineer with 5 years of experience
    - I've built a few AI projects in the past
    - I'm looking to start a new project either building an AI blog posting assistant as a SaaS or building a keyboard shortcut application as a desktop app.
    - 4 years of NextJS experience, 2 years of Python experience, 1 year of AI experience
    """

    # Create a list of tasks to run in parallel
    answers = await asyncio.gather(
        analyze_decision("software engineer", context),
        analyze_decision("software engineer", context),
        analyze_decision("software engineer", context),
    )
    final_answer = await aggregate_to_a_final_answer("career coach", "the user is looking for a new job", answers)
    print(final_answer)

asyncio.run(main())

Based on the analysis of your two project ideas—an **AI Blog Posting Assistant (SaaS)** and a **Keyboard Shortcut Application (Desktop App)**—here’s a consolidated recommendation to help you move forward in your career transition:

### Project Options Overview

1. **AI Blog Posting Assistant (SaaS)**
   - **Pros:**
     - Leverages your existing AI and web development skills, particularly with Next.js.
     - The SaaS model offers potential for recurring revenue and scalability.
     - Growing demand for AI-driven content creation tools can lead to a larger user base.
   - **Cons:**
     - High competition in the market with established players.
     - Requires ongoing maintenance, customer support, and marketing efforts.

2. **Keyboard Shortcut Application (Desktop App)**
   - **Pros:**
     - Potentially simpler and quicker to develop, allowing for faster market entry.
     - A niche market may have less competition, especially if you focus on unique features.
     - You can utilize 